In [2]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from collections import Counter
import heapq
import nltk
from nltk.corpus import stopwords
import itertools
%matplotlib inline

In [1]:
!pip install pattern
!pip install gensim

The directory '/home/shevkunov/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/shevkunov/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/shevkunov/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/shevkunov/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shevkunov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
english_stopwords = set(stopwords.words('english'))

In [5]:
dataset = fetch_20newsgroups(
    remove=('headers', 'footers', 'quotes')
)

In [6]:
import gensim 

In [8]:
pured_documents = []
for i, doc in enumerate(dataset.data):
    tokens = gensim.utils.lemmatize(doc)
    document = []
    for token in tokens:
        word = token.split('/')[0]
        if word not in english_stopwords:
            document.append(word)
    pured_documents.append(document)    
    if i % 500 == 0:
        print 'Processed: ', i, 'documents from', len(dataset.data)

Processed:  0 documents from 11314
Processed:  500 documents from 11314
Processed:  1000 documents from 11314
Processed:  1500 documents from 11314
Processed:  2000 documents from 11314
Processed:  2500 documents from 11314
Processed:  3000 documents from 11314
Processed:  3500 documents from 11314
Processed:  4000 documents from 11314
Processed:  4500 documents from 11314
Processed:  5000 documents from 11314
Processed:  5500 documents from 11314
Processed:  6000 documents from 11314
Processed:  6500 documents from 11314
Processed:  7000 documents from 11314
Processed:  7500 documents from 11314
Processed:  8000 documents from 11314
Processed:  8500 documents from 11314
Processed:  9000 documents from 11314
Processed:  9500 documents from 11314
Processed:  10000 documents from 11314
Processed:  10500 documents from 11314
Processed:  11000 documents from 11314


In [27]:
text = nltk.Text(list(itertools.chain.from_iterable(pured_documents)))


In [44]:
class NltkHatPlayer(object):
    def __init__(self):
        pass
    
    def guess(self, words):
        candidates = Counter()
        for word in words:
            for w in self.explain(word)[:5]:
                candidates[w] += 1
        for word in words:
            if word in candidates:
                del candidates[word]
        return [x for x, _ in candidates.most_common(len(candidates))]
    
    def explain(self, word):
        text.common_contexts(word,num=5);
        return text._word_context_index.similar_words(word)
    
    def guess_result(self, position):
        pass
    
    def explanation_result(self, position):
        pass

In [45]:
from gensim.models import Word2Vec

In [46]:
model = Word2Vec(pured_documents, size=100, window=5, min_count=5, workers=4)

In [47]:
class GensimHatPlayer(object):
    def __init__(self):
        pass
    
    def guess(self, words):
        return [x for x, _ in model.most_similar(positive=words)]
    
    def explain(self, word):
        return [x for x, _ in model.most_similar(positive=[word])]
    
    def guess_result(self, position):
        pass
    
    def explanation_result(self, position):
        pass

In [48]:
words_universe = model.vocab.keys()
words_universe_set = set(model.vocab.keys())

In [49]:
def calc_score(player1, player2, word):
    explanation = [x for x in player1.explain(word) if x in words_universe_set][:10]
    exp_score = 0.
    if word in explanation:
        exp_score -= 1.0
    guess_score = 0.
    if len(explanation) > 0:
        for pref in xrange(10):
            guess = player2.guess(explanation[:pref + 1])[:10]
            try:
                pos = guess.index(word)
                player1.explanation_result(pos)
                player2.guess_result(pos)
                guess_score += 0.9 ** pos
                exp_score += 0.9 ** pos
            except ValueError:
                player1.explanation_result(None)
                player2.guess_result(None)
    return exp_score, guess_score


def play(player1, player2, rounds, seed=42):
    random_gen = np.random.RandomState(seed)
    player1_exp_score = 0.
    player1_guess_score = 0.
    player2_exp_score = 0.
    player2_guess_score = 0.
    for _ in xrange(rounds):
        word = random_gen.choice(words_universe)
        player1_exp, player2_guess = calc_score(player1, player2, word)
        player2_exp, player1_guess = calc_score(player2, player1, word)
        player1_explanation = player1.explain(word)
        player2_explanation = player1.explain(word)
        
        player1_exp_score += player1_exp
        player2_exp_score += player2_exp
        player1_guess_score += player1_guess
        player2_guess_score += player2_guess
        
    return (
        (player1_exp_score, player1_guess_score, player1_guess_score + player1_exp_score),
        (player2_exp_score, player2_guess_score, player2_guess_score + player2_exp_score)
    )

In [50]:
%%time
play(GensimHatPlayer(), GensimHatPlayer(), 1000)

CPU times: user 1min 53s, sys: 1min 36s, total: 3min 29s
Wall time: 1min 8s


((7881.156060668992, 7881.156060668992, 15762.312121337984),
 (7881.156060668992, 7881.156060668992, 15762.312121337984))

In [51]:
%%time
play(NltkHatPlayer(), GensimHatPlayer(), 1000)

(u'The following word(s) were not found:', u's l i d e')
(u'The following word(s) were not found:', u'w r i s t')
(u'The following word(s) were not found:', u'w r i s t')
(u'The following word(s) were not found:', u'l a n e')
(u'The following word(s) were not found:', u'w r i s t')
(u'The following word(s) were not found:', u'l a n e')
(u'The following word(s) were not found:', u'f l a s h')
(u'The following word(s) were not found:', u'w r i s t')
(u'The following word(s) were not found:', u'l a n e')
(u'The following word(s) were not found:', u'f l a s h')
(u'The following word(s) were not found:', u'f a d e')
(u'The following word(s) were not found:', u'w r i s t')
(u'The following word(s) were not found:', u'l a n e')
(u'The following word(s) were not found:', u'f l a s h')
(u'The following word(s) were not found:', u'f a d e')
(u'The following word(s) were not found:', u'p i l l i o n')
(u'The following word(s) were not found:', u'w r i s t')
(u'The following word(s) were not found

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



(u'The following word(s) were not found:', u'p e t e r s o n')
(u'The following word(s) were not found:', u'd e c w r l')
(u'The following word(s) were not found:', u'd e c w r l')
(u'The following word(s) were not found:', u'c o u r t e s y')
(u'The following word(s) were not found:', u'd e c w r l')
(u'The following word(s) were not found:', u'c o u r t e s y')
(u'The following word(s) were not found:', u'b o o g i e')
(u'The following word(s) were not found:', u'd e c w r l')
(u'The following word(s) were not found:', u'c o u r t e s y')
(u'The following word(s) were not found:', u'b o o g i e')
(u'The following word(s) were not found:', u'k a r l')
(u'The following word(s) were not found:', u'd e c w r l')
(u'The following word(s) were not found:', u'c o u r t e s y')
(u'The following word(s) were not found:', u'b o o g i e')
(u'The following word(s) were not found:', u'k a r l')
(u'The following word(s) were not found:', u'f t p m a i l')
(u'The following word(s) were not found:',

IndexError: string index out of range

In [ ]:
%%time
play(NltkHatPlayer(), NltkHatPlayer(), 1000)